# Data Source: Failory

In [18]:
# STarting off by importing necessary libraries and listing essentials like Urls and Headers

import requests, time, re
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin

base_url = "https://www.failory.com"
list_url = "https://www.failory.com/startups"
headers = {"User-Agent": "Mozilla/5.0 (compatible; EU-Startup-Analysis/1.0)"}

In [19]:
# Intial test to see if the website can be scraped

resp = requests.get(list_url, headers=headers, timeout=30)
resp.raise_for_status()

soup = BeautifulSoup(resp.text, "html.parser")
soup.title

<title>The Largest Database of Startups (+14,000)</title>

In [20]:
# Checking all the links we can access under startups page.

links = []

# typical card links live under /startup/ or /startups/
for a in soup.select("a[href]"):
    href = a["href"]
    if href.startswith("/startup/") or href.startswith("/startups/"):
        full = urljoin(BASE, href)
        if full not in links:
            links.append(full)

print("Discovered detail links:", len(links))
links[:10]


Discovered detail links: 267


['https://www.failory.com/startups/united-states',
 'https://www.failory.com/startups/colombia',
 'https://www.failory.com/startups/greece',
 'https://www.failory.com/startups/bangladesh',
 'https://www.failory.com/startups/france',
 'https://www.failory.com/startups/pakistan',
 'https://www.failory.com/startups/canada',
 'https://www.failory.com/startups/romania',
 'https://www.failory.com/startups/new-zealand',
 'https://www.failory.com/startups/indonesia']

##### Here, we can see that the website is structured as country for a single link. Lets try to explore further and then decide on a scraping strategy

In [ ]:
# Trying out a preview of rows for each country page to see the results

rows_preview = []

for link in links[:3]:
    print("Fetching:", link)
    r = requests.get(link, headers=headers, timeout=30)
    r.raise_for_status()
    doc = BeautifulSoup(r.text, "html.parser")

    # name
    name_tag = doc.find(["h1","h2"])
    name = name_tag.get_text(strip=True) if name_tag else None

    # short description
    meta_desc = doc.find("meta", {"name":"description"})
    if meta_desc and meta_desc.get("content"):
        short_desc = meta_desc["content"].strip()
    else:
        p = name_tag.find_next("p") if name_tag else None
        short_desc = p.get_text(" ", strip=True) if p else None

    # full text for simple heuristics
    text = doc.get_text(" ", strip=True)

    # trying out a very rough country heuristic
    country = None
    for pat in [
        r"based in ([A-Z][A-Za-z \-]{2,50})",
        r"based in,? ([A-Z][A-Za-z \-]{2,50})",
        r"based in ([A-Z][A-Za-z \-]{2,50}),",
        r"Founded in ([A-Z][A-Za-z \-]{2,50})"
    ]:
        m = re.search(pat, text, re.I)
        if m:
            country = m.group(1).strip()
            break

    # funding amount heuristic (currency + number + optional suffix)
    funding_amount = None
    m_amt = re.search(r"(€|£|\$)\s?([\d\.,]+)\s?(k|m|bn|b)?", text, re.I)
    if m_amt:
        funding_amount = m_amt.group(0)

    # year near "raised/funded/in"
    funding_year = None
    m_year = re.search(r"(raised|funded|in)\s+(20[0-3]\d)", text, re.I)
    if m_year:
        funding_year = m_year.group(2)

    # investors heuristic
    investors = None
    m_inv = re.search(r"([\d,]{1,6})\s+(investors|backers)", text, re.I)
    if m_inv:
        investors = m_inv.group(1).replace(",", "")

    rows_preview.append({
        "url": link,
        "name": name,
        "short_description": short_desc,
        "country": country,
        "funding_amount": funding_amount,
        "funding_year": funding_year,
        "investors": investors
    })

    time.sleep(1.5)  #  wait time for caution

pd.DataFrame(rows_preview)


Fetching: https://www.failory.com/startups/united-states
Fetching: https://www.failory.com/startups/colombia
Fetching: https://www.failory.com/startups/greece


,url,name,short_description,country,funding_amount,funding_year,investors
0,https://www.failory.com/startups/united-states,Top 300 Startups in United States in 2024,United States is home to hundreds of tech star...,the United States,$97,2024,None
1,https://www.failory.com/startups/colombia,Top 36 Startups in Colombia in 2024,Colombia is home to hundreds of tech startups....,None,$97,2024,None
2,https://www.failory.com/startups/greece,Top 78 Startups in Greece in 2024,Greece is home to hundreds of tech startups. I...,Silicon Valley Here are,$97,2024,None


#### Even though the scraping is successful, the data is obviously wrong due to the way we scraped. I will now try to scrape a single country and see how it goes

In [22]:
# Example: running for a single country first

country = "austria" 
url = f"{base_url}/startups/{country}"
url

'https://www.failory.com/startups/austria'

In [23]:
# Checking the title to see if scraping is successful
resp = requests.get(url, headers=headers, timeout=45)
resp.raise_for_status()
soup = BeautifulSoup(resp.text, "html.parser")
soup.title


<title>Top 108 Startups in Austria in 2024</title>

In [24]:
# Printing out the raw text to test scraped data
text = soup.get_text("\n", strip=True)
print(text[:1200])

Top 108 Startups in Austria in 2024
Failory
Interviews
Blog
Graveyard
Products
Subscribe â
Download Our List of The Top 100 Accelerators & Incubators
This free sheet contains 100 accelerators and incubators you can apply to today, along with information about the industries they generally invest in.
Thank you! Your submission has been received!
Oops! Something went wrong while submitting the form.
Download The List of theÂ 100 Highest-Valued Unicorns
This free sheet contains all the information about the top 100 unicorns, including their valuation, HQ's location, founded year, name of founders, funding amount and number of employees.
Thank you! Your submission has been received!
Oops! Something went wrong while submitting the form.
Get Free Access to The Founder's Handbook
This free Notion document contains the best 100+ resources you need for building a successful startup, divided in 4 categories:Â Fundraising, People, Product, andÂ Growth.
Thank you! Your submission has been receiv

In [25]:
# The data of startups is listed in a single country page with a specific format so we do this way.
# Creating parts and looping through them. parts looks like: ["prefix", "1", "Bitpanda\n....", "2", "Another...\n....", ...]

parts = re.split(r"\n(\d+)\)\s+", text)
len(parts)

217

In [26]:
# Rebuild as list of (rank, block_text)
blocks = []
for i in range(1, len(parts), 2):
    rank = parts[i]
    block = parts[i+1] if (i+1) < len(parts) else ""
    blocks.append((rank, block))

len(blocks), blocks[0][0], blocks[0][1][:200]


(108,
 '1',
 'Bitpanda\nBitpanda is a digital asset exchange that makes it easy to purchase and sell bitcoins.\nDetails of the startup:\nState: Wien\nCity: Vienna\nStarted in: 2014\nFounders: Christian Trummer, Eric Demu')

#### With the blocks now successfully showing the number of startups for a country, let us now proceed with the scraping.

In [27]:
preview_rows = []
for rank, block in blocks[:3]:
    lines = [ln.strip() for ln in block.split("\n") if ln.strip()]
    # Company name = first non-empty line
    name = lines[0] if lines else None

    # Description = lines until we hit 'Details of the startup:'
    desc_lines = []
    details_idx = None
    for idx, ln in enumerate(lines[1:], start=1):
        if ln.lower().startswith("details of the startup"):
            details_idx = idx
            break
        desc_lines.append(ln)
    description = " ".join(desc_lines).strip() if desc_lines else None

    # Details: parse key-value pairs from subsequent lines
    details_text = "\n".join(lines[details_idx+1:]) if details_idx is not None else ""
    # Normalize whitespaces
    details_text = re.sub(r"[ \t]+", " ", details_text)

    # Prepare holders
    state = city = started_in = founders = industries = employees = None
    funding_raw = funding_num = funding_cur = funding_rounds = None
    investors_count = None
    investors_list = None

    # Simple extractors for all fields listed on the website.
    m = re.search(r"\bState:\s*(.+)", details_text, re.I);           state = m.group(1).strip() if m else None
    m = re.search(r"\bCity:\s*(.+)", details_text, re.I);            city = m.group(1).strip() if m else None
    m = re.search(r"\bStarted in:\s*(\d{4})", details_text, re.I);   started_in = m.group(1) if m else None
    m = re.search(r"\bFounders:\s*(.+)", details_text, re.I);        founders = m.group(1).strip() if m else None
    m = re.search(r"\bIndustries:\s*(.+)", details_text, re.I);      industries = m.group(1).strip() if m else None
    m = re.search(r"\bNumber of employees:\s*(.+)", details_text, re.I); employees = m.group(1).strip() if m else None
    m = re.search(r"\bFunding:\s*([€£\$]\s?[\d,\.]+(?:\s?(k|m|bn|b))?)", details_text, re.I)
    if m:
        funding_raw = m.group(1)

    m = re.search(r"\bFunding rounds:\s*(\d+)", details_text, re.I);  funding_rounds = m.group(1) if m else None

    # Investors pattern like: "Number of investors: 6 (DST Global, Wintermute Trading, ...)"
    m = re.search(r"\bNumber of investors:\s*(\d+)\s*\((.*?)\)", details_text, re.I)
    if m:
        investors_count = m.group(1)
        investors_list = [x.strip() for x in m.group(2).split(",")]
    else:
        # if no list in parentheses, try just the count
        m = re.search(r"\bNumber of investors:\s*(\d+)", details_text, re.I)
        if m:
            investors_count = m.group(1)

    preview_rows.append({
        "rank": int(rank),
        "country": country.replace("-", " ").title(),
        "name": name,
        "description": description,
        "state": state,
        "city": city,
        "started_in": started_in,
        "founders": founders,
        "industries": industries,
        "employees": employees,
        "funding_raw": funding_raw,
        "funding_rounds": funding_rounds,
        "investors_count": investors_count,
        "investors_list": investors_list,
        "source_url": url
    })

pd.DataFrame(preview_rows)


,rank,country,name,description,state,city,started_in,founders,industries,employees,funding_raw,funding_rounds,investors_count,investors_list,source_url
0,1,Austria,Bitpanda,Bitpanda is a digital asset exchange that make...,Wien,Vienna,2014,"Christian Trummer, Eric Demuth, Paul Klanschek","Bitcoin, Blockchain, Cryptocurrency, FinTech",251-500,"$283,013,472",5,6,"[DST Global, Wintermute Trading, Speedinvest, ...",https://www.failory.com/startups/austria
1,2,Austria,Tractive,Tractive is a company that creates GPS trackin...,Oberosterreich,Pasching,2012,"Michael Hurnaus, Michael Lettner, Michael Tsch...","GPS, Pet, Software, Wearables",51-100,"$37,687,997",5,5,"[Guidepost Growth Equity, Monkfish Equity, Har...",https://www.failory.com/startups/austria
2,3,Austria,GoStudent,"In a virtual learning environment, GoStudent i...",Wien,Vienna,2016,"Felix Ohswald, Gregor MÃ¼ller, Moritz Ohswald","Apps, Continuing Education, EdTech, Education",251-500,"$100,662,844",6,7,"[DN Capital, Coatue, Speedinvest, Left Lane Ca...",https://www.failory.com/startups/austria


#### The preview records look successful so we can proceed with the full scraping of the country.

In [28]:
rows = []
for rank, block in blocks:
    lines = [ln.strip() for ln in block.split("\n") if ln.strip()]
    if not lines:
        continue

    name = lines[0]

    desc_lines = []
    details_idx = None
    for idx, ln in enumerate(lines[1:], start=1):
        if ln.lower().startswith("details of the startup"):
            details_idx = idx
            break
        desc_lines.append(ln)
    description = " ".join(desc_lines).strip() if desc_lines else None

    details_text = "\n".join(lines[details_idx+1:]) if details_idx is not None else ""
    details_text = re.sub(r"[ \t]+", " ", details_text)

    state = city = started_in = founders = industries = employees = None
    funding_raw = funding_rounds = None
    investors_count = None
    investors_list = None

    m = re.search(r"\bState:\s*(.+)", details_text, re.I);           state = m.group(1).strip() if m else None
    m = re.search(r"\bCity:\s*(.+)", details_text, re.I);            city = m.group(1).strip() if m else None
    m = re.search(r"\bStarted in:\s*(\d{4})", details_text, re.I);   started_in = m.group(1) if m else None
    m = re.search(r"\bFounders:\s*(.+)", details_text, re.I);        founders = m.group(1).strip() if m else None
    m = re.search(r"\bIndustries:\s*(.+)", details_text, re.I);      industries = m.group(1).strip() if m else None
    m = re.search(r"\bNumber of employees:\s*(.+)", details_text, re.I); employees = m.group(1).strip() if m else None
    m = re.search(r"\bFunding:\s*([€£\$]\s?[\d,\.]+(?:\s?(k|m|bn|b))?)", details_text, re.I)
    if m:
        funding_raw = m.group(1)
    m = re.search(r"\bFunding rounds:\s*(\d+)", details_text, re.I);  funding_rounds = m.group(1) if m else None

    m = re.search(r"\bNumber of investors:\s*(\d+)\s*\((.*?)\)", details_text, re.I)
    if m:
        investors_count = m.group(1)
        investors_list = [x.strip() for x in m.group(2).split(",")]
    else:
        m = re.search(r"\bNumber of investors:\s*(\d+)", details_text, re.I)
        if m:
            investors_count = m.group(1)

    rows.append({
        "rank": int(rank),
        "country": country.replace("-", " ").title(),
        "name": name,
        "description": description,
        "state": state,
        "city": city,
        "started_in": started_in,
        "founders": founders,
        "industries": industries,
        "employees": employees,
        "funding_raw": funding_raw,
        "funding_rounds": funding_rounds,
        "investors_count": investors_count,
        "investors_list": investors_list,
        "source_url": url
    })

df_country = pd.DataFrame(rows).sort_values("rank").reset_index(drop=True)
df_country.shape


(108, 15)

In [29]:
df_country.head()

,rank,country,name,description,state,city,started_in,founders,industries,employees,funding_raw,funding_rounds,investors_count,investors_list,source_url
0,1,Austria,Bitpanda,Bitpanda is a digital asset exchange that make...,Wien,Vienna,2014,"Christian Trummer, Eric Demuth, Paul Klanschek","Bitcoin, Blockchain, Cryptocurrency, FinTech",251-500,"$283,013,472",5,6,"[DST Global, Wintermute Trading, Speedinvest, ...",https://www.failory.com/startups/austria
1,2,Austria,Tractive,Tractive is a company that creates GPS trackin...,Oberosterreich,Pasching,2012,"Michael Hurnaus, Michael Lettner, Michael Tsch...","GPS, Pet, Software, Wearables",51-100,"$37,687,997",5,5,"[Guidepost Growth Equity, Monkfish Equity, Har...",https://www.failory.com/startups/austria
2,3,Austria,GoStudent,"In a virtual learning environment, GoStudent i...",Wien,Vienna,2016,"Felix Ohswald, Gregor MÃ¼ller, Moritz Ohswald","Apps, Continuing Education, EdTech, Education",251-500,"$100,662,844",6,7,"[DN Capital, Coatue, Speedinvest, Left Lane Ca...",https://www.failory.com/startups/austria
3,4,Austria,Allcyte,Allcyte is a biotech start-up focused on patie...,Wien,Vienna,2017,"Berend Snijder, Giulio Superti-Furga, Gregory ...","Biotechnology, Health Diagnostics, Machine Lea...",11-50,"$6,000,000",2,5,"[42CAP, Air Street Capital, PUSH Ventures, Ami...",https://www.failory.com/startups/austria
4,5,Austria,Adverity,Adverity is a data-driven marketing analytics ...,Wien,Vienna,2015,"Alexander IgelsbÃ¶ck, Andreas GlÃ¤nzer, Martin...","Advertising, Analytics, Artificial Intelligenc...",101-250,"$46,298,685",4,7,"[Sapphire Ventures, Speedinvest, Felix Capital...",https://www.failory.com/startups/austria


#### The data looks well scraped and placed correctly so let us now do the same for all european countries at once.

In [31]:
eu_countries = [
    "austria","belgium","bulgaria","croatia","cyprus","czech-republic","denmark","estonia","finland",
    "france","germany","greece","hungary","ireland","italy","latvia","lithuania","luxembourg","malta",
    "netherlands","poland","portugal","romania","slovakia","slovenia","spain","sweden",
    "united-kingdom","switzerland","norway","iceland"
]

all_rows = []

for slug in eu_countries:
    url = f"{base_url}/startups/{slug}"
    print("Country:", slug, "=>", url)
    try:
        resp = requests.get(url, headers=headers, timeout=45); resp.raise_for_status()
    except Exception as e:
        print("  failed:", e); time.sleep(2); continue

    soup = BeautifulSoup(resp.text, "html.parser")
    text = soup.get_text("\n", strip=True)
    parts = re.split(r"\n(\d+)\)\s+", text)
    blocks = []
    for i in range(1, len(parts), 2):
        rank = parts[i]
        block = parts[i+1] if (i+1) < len(parts) else ""
        blocks.append((rank, block))

    for rank, block in blocks:
        lines = [ln.strip() for ln in block.split("\n") if ln.strip()]
        if not lines: 
            continue
        name = lines[0]

        desc_lines = []
        details_idx = None
        for idx, ln in enumerate(lines[1:], start=1):
            if ln.lower().startswith("details of the startup"):
                details_idx = idx; break
            desc_lines.append(ln)
        description = " ".join(desc_lines).strip() if desc_lines else None

        details_text = "\n".join(lines[details_idx+1:]) if details_idx is not None else ""
        details_text = re.sub(r"[ \t]+", " ", details_text)

        state = city = started_in = founders = industries = employees = None
        funding_raw = funding_rounds = None
        investors_count = None
        investors_list = None

        m = re.search(r"\bState:\s*(.+)", details_text, re.I);           state = m.group(1).strip() if m else None
        m = re.search(r"\bCity:\s*(.+)", details_text, re.I);            city = m.group(1).strip() if m else None
        m = re.search(r"\bStarted in:\s*(\d{4})", details_text, re.I);   started_in = m.group(1) if m else None
        m = re.search(r"\bFounders:\s*(.+)", details_text, re.I);        founders = m.group(1).strip() if m else None
        m = re.search(r"\bIndustries:\s*(.+)", details_text, re.I);      industries = m.group(1).strip() if m else None
        m = re.search(r"\bNumber of employees:\s*(.+)", details_text, re.I); employees = m.group(1).strip() if m else None
        m = re.search(r"\bFunding:\s*([€£\$]\s?[\d,\.]+(?:\s?(k|m|bn|b))?)", details_text, re.I)
        if m:
            funding_raw = m.group(1)
        m = re.search(r"\bFunding rounds:\s*(\d+)", details_text, re.I);  funding_rounds = m.group(1) if m else None

        m = re.search(r"\bNumber of investors:\s*(\d+)\s*\((.*?)\)", details_text, re.I)
        if m:
            investors_count = m.group(1)
            investors_list = [x.strip() for x in m.group(2).split(",")]
        else:
            m = re.search(r"\bNumber of investors:\s*(\d+)", details_text, re.I)
            if m:
                investors_count = m.group(1)

        all_rows.append({
            "rank": int(rank),
            "country": slug.replace("-", " ").title(),
            "name": name,
            "description": description,
            "state": state,
            "city": city,
            "started_in": started_in,
            "founders": founders,
            "industries": industries,
            "employees": employees,
            "funding_raw": funding_raw,
            "funding_rounds": funding_rounds,
            "investors_count": investors_count,
            "investors_list": investors_list,
            "source_url": url
        })

    # polite pause per country
    time.sleep(2)

df_failory = pd.DataFrame(all_rows).sort_values(["country","rank"]).reset_index(drop=True)
df_failory.shape


Country: austria => https://www.failory.com/startups/austria


Country: belgium => https://www.failory.com/startups/belgium
Country: bulgaria => https://www.failory.com/startups/bulgaria
Country: croatia => https://www.failory.com/startups/croatia
Country: cyprus => https://www.failory.com/startups/cyprus
Country: czech-republic => https://www.failory.com/startups/czech-republic
Country: denmark => https://www.failory.com/startups/denmark
Country: estonia => https://www.failory.com/startups/estonia
Country: finland => https://www.failory.com/startups/finland
Country: france => https://www.failory.com/startups/france
Country: germany => https://www.failory.com/startups/germany
Country: greece => https://www.failory.com/startups/greece
Country: hungary => https://www.failory.com/startups/hungary
Country: ireland => https://www.failory.com/startups/ireland
Country: italy => https://www.failory.com/startups/italy
Country: latvia => https://www.failory.com/startups/latvia
Country: lithuania => https://www.failory.com/startups/lithuania
Country: luxembo

(4803, 15)

##### Judging from the output, netherlands failed. Upon inspecting the website, I came to the realization that the country is listed as 'The Netherlands' instead of Netherlands. So, let us change the list item to 'the-netherlands'

In [32]:
eu_countries = [
    "austria","belgium","bulgaria","croatia","cyprus","czech-republic","denmark","estonia","finland",
    "france","germany","greece","hungary","ireland","italy","latvia","lithuania","luxembourg","malta",
    "the-netherlands","poland","portugal","romania","slovakia","slovenia","spain","sweden",
    "united-kingdom","switzerland","norway","iceland"
]

all_rows = []

for slug in eu_countries:
    url = f"{base_url}/startups/{slug}"
    print("Country:", slug, "=>", url)
    try:
        resp = requests.get(url, headers=headers, timeout=45); resp.raise_for_status()
    except Exception as e:
        print("  failed:", e); time.sleep(2); continue

    soup = BeautifulSoup(resp.text, "html.parser")
    text = soup.get_text("\n", strip=True)
    parts = re.split(r"\n(\d+)\)\s+", text)
    blocks = []
    for i in range(1, len(parts), 2):
        rank = parts[i]
        block = parts[i+1] if (i+1) < len(parts) else ""
        blocks.append((rank, block))

    for rank, block in blocks:
        lines = [ln.strip() for ln in block.split("\n") if ln.strip()]
        if not lines: 
            continue
        name = lines[0]

        desc_lines = []
        details_idx = None
        for idx, ln in enumerate(lines[1:], start=1):
            if ln.lower().startswith("details of the startup"):
                details_idx = idx; break
            desc_lines.append(ln)
        description = " ".join(desc_lines).strip() if desc_lines else None

        details_text = "\n".join(lines[details_idx+1:]) if details_idx is not None else ""
        details_text = re.sub(r"[ \t]+", " ", details_text)

        state = city = started_in = founders = industries = employees = None
        funding_raw = funding_rounds = None
        investors_count = None
        investors_list = None

        m = re.search(r"\bState:\s*(.+)", details_text, re.I);           state = m.group(1).strip() if m else None
        m = re.search(r"\bCity:\s*(.+)", details_text, re.I);            city = m.group(1).strip() if m else None
        m = re.search(r"\bStarted in:\s*(\d{4})", details_text, re.I);   started_in = m.group(1) if m else None
        m = re.search(r"\bFounders:\s*(.+)", details_text, re.I);        founders = m.group(1).strip() if m else None
        m = re.search(r"\bIndustries:\s*(.+)", details_text, re.I);      industries = m.group(1).strip() if m else None
        m = re.search(r"\bNumber of employees:\s*(.+)", details_text, re.I); employees = m.group(1).strip() if m else None
        m = re.search(r"\bFunding:\s*([€£\$]\s?[\d,\.]+(?:\s?(k|m|bn|b))?)", details_text, re.I)
        if m:
            funding_raw = m.group(1)
        m = re.search(r"\bFunding rounds:\s*(\d+)", details_text, re.I);  funding_rounds = m.group(1) if m else None

        m = re.search(r"\bNumber of investors:\s*(\d+)\s*\((.*?)\)", details_text, re.I)
        if m:
            investors_count = m.group(1)
            investors_list = [x.strip() for x in m.group(2).split(",")]
        else:
            m = re.search(r"\bNumber of investors:\s*(\d+)", details_text, re.I)
            if m:
                investors_count = m.group(1)

        all_rows.append({
            "rank": int(rank),
            "country": slug.replace("-", " ").title(),
            "name": name,
            "description": description,
            "state": state,
            "city": city,
            "started_in": started_in,
            "founders": founders,
            "industries": industries,
            "employees": employees,
            "funding_raw": funding_raw,
            "funding_rounds": funding_rounds,
            "investors_count": investors_count,
            "investors_list": investors_list,
            "source_url": url
        })

    # polite pause per country
    time.sleep(2)

df_failory = pd.DataFrame(all_rows).sort_values(["country","rank"]).reset_index(drop=True)
df_failory.shape


Country: austria => https://www.failory.com/startups/austria
Country: belgium => https://www.failory.com/startups/belgium
Country: bulgaria => https://www.failory.com/startups/bulgaria
Country: croatia => https://www.failory.com/startups/croatia
Country: cyprus => https://www.failory.com/startups/cyprus
Country: czech-republic => https://www.failory.com/startups/czech-republic
Country: denmark => https://www.failory.com/startups/denmark
Country: estonia => https://www.failory.com/startups/estonia
Country: finland => https://www.failory.com/startups/finland
Country: france => https://www.failory.com/startups/france
Country: germany => https://www.failory.com/startups/germany
Country: greece => https://www.failory.com/startups/greece


Country: hungary => https://www.failory.com/startups/hungary
Country: ireland => https://www.failory.com/startups/ireland
Country: italy => https://www.failory.com/startups/italy
Country: latvia => https://www.failory.com/startups/latvia
Country: lithuania => https://www.failory.com/startups/lithuania
Country: luxembourg => https://www.failory.com/startups/luxembourg
Country: malta => https://www.failory.com/startups/malta
Country: the-netherlands => https://www.failory.com/startups/the-netherlands
Country: poland => https://www.failory.com/startups/poland
Country: portugal => https://www.failory.com/startups/portugal
Country: romania => https://www.failory.com/startups/romania
Country: slovakia => https://www.failory.com/startups/slovakia
Country: slovenia => https://www.failory.com/startups/slovenia
Country: spain => https://www.failory.com/startups/spain
Country: sweden => https://www.failory.com/startups/sweden
Country: united-kingdom => https://www.failory.com/startups/united-king

(5103, 15)

In [33]:
# Writing to a csv file for further cleaning at next step
df_failory.to_csv('failory_eu_companies_raw.csv', index=None)